In [2]:
!wget https://fling.seas.upenn.edu/~cis680/wiki/Projects/Fall2017/celeba.zip

--2018-12-01 17:38:51--  https://fling.seas.upenn.edu/~cis680/wiki/Projects/Fall2017/celeba.zip
Resolving fling.seas.upenn.edu (fling.seas.upenn.edu)... 158.130.68.16
Connecting to fling.seas.upenn.edu (fling.seas.upenn.edu)|158.130.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 574859470 (548M) [application/zip]
Saving to: ‘celeba.zip’

celeba.zip          100%[===================>] 548.23M  44.9MB/s    in 12s     

2018-12-01 17:39:03 (47.2 MB/s) - ‘celeba.zip’ saved [574859470/574859470]

